# Exploratory Analysis of Search and MDP Algorithms

This notebook analyses the performance of different **search** and **MDP** algorithms across different maze sizes.

We evaluate execution time, memory usage, path length, convergence rate, and nodes expanded.

1. Identify patterns in algorithm performance
2. Compare search algorithms (DFS, BFS, A*) and MDP algorithms (Value Iteration, Policy Iteration)
3. Detect outliers in performance metrics
4. Perform statistical comparisons to determine differences

In [1]:

import pandas as pd
import scipy.stats as stats
import seaborn as sns

# Load the performance data
df = pd.read_csv("../data/results/performance_metrics.csv")

# # Identify Non-Numeric Values Before Processing
numeric_columns = ["Execution Time", "Memory Usage", "Path Length", "Convergence Rate", "Nodes Expanded"]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # Convert all to numeric, set invalid values to NaN

# Drop Rows Where All Numeric Metrics Are NaN
df_cleaned = df.dropna(subset=numeric_columns, how='all')

# Check if dataset is still empty after dropping
if df_cleaned.empty:
    print("Warning: No valid numeric data after cleaning.")


**Data Cleaning Summary:**
- Converted all numerical columns to float
- Dropped rows with entirely missing numeric values
- If no valid data remains, a warning is displayed

In [2]:

# Dataset Overview
print("\nDataset Information:")
print(df_cleaned.info())


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Maze ID                219 non-null    object 
 1   Timestamp              219 non-null    object 
 2   Maze Size              219 non-null    int64  
 3   Algorithm Type         219 non-null    object 
 4   Algorithm              219 non-null    object 
 5   Run Index              219 non-null    int64  
 6   Execution Time         219 non-null    float64
 7   Memory Usage           218 non-null    float64
 8   Path Length            219 non-null    int64  
 9   Convergence Rate       59 non-null     float64
 10  Optimality             219 non-null    int64  
 11  Nodes Expanded         160 non-null    float64
 12  Invalid Maze Attempts  219 non-null    int64  
 13  Failed Paths           219 non-null    int64  
dtypes: float64(4), int64(6), object(4)
m

**Dataset Overview:**
- This displays the data types and number of non-null values per column
- Ensures that "Maze Size" and categorical columns are properly formatted


In [3]:

print("\nSummary Statistics:")
display(df_cleaned.describe())



Summary Statistics:


,Maze Size,Run Index,Execution Time,Memory Usage,Path Length,Convergence Rate,Optimality,Nodes Expanded,Invalid Maze Attempts,Failed Paths
count,219.000000,219.000000,219.000000,2.180000e+02,219.000000,59.000000,219.0,160.000000,219.000000,219.000000
mean,173.515982,3.009132,0.955978,5.573722e+06,6447.812785,148.745763,1.0,35808.656250,1.872146,0.479452
std,259.623428,1.414184,2.486665,1.426019e+07,15519.887665,82.822118,0.0,86952.553599,2.938529,1.459951
min,10.000000,1.000000,0.000011,1.256000e+03,4.000000,9.000000,1.0,6.000000,0.000000,0.000000
25%,25.000000,2.000000,0.000447,2.376000e+04,34.000000,113.500000,1.0,59.500000,0.000000,0.000000
50%,75.000000,3.000000,0.013893,3.489520e+05,220.000000,133.000000,1.0,1362.500000,1.000000,0.000000
75%,200.000000,4.000000,0.450213,3.905728e+06,2450.500000,238.500000,1.0,14703.500000,3.000000,0.000000
max,1000.000000,5.000000,13.543401,7.550696e+07,64495.000000,265.000000,1.0,374714.000000,14.000000,9.000000


**Summary Statistics:**
- Shows mean, standard deviation, and range of key performance metrics
- Helps identify potential outliers and data trends

In [4]:

# Check unique maze sizes tested
unique_sizes = sorted(df_cleaned["Maze Size"].unique().astype(int).tolist())
print("\nUnique Maze Sizes Tested:", unique_sizes)


Unique Maze Sizes Tested: [10, 25, 50, 75, 100, 200, 300, 500, 1000]


**Maze Sizes Overview:**
- Lists the different maze sizes tested to ensure variation in dataset.


In [5]:
# Convert Data Types for Proper Processing
df_cleaned["Algorithm Type"] = df_cleaned["Algorithm Type"].astype("category")
df_cleaned["Algorithm"] = df_cleaned["Algorithm"].astype("category")
df_cleaned["Maze Size"] = df_cleaned["Maze Size"].astype(int)
df_cleaned["Success"] = df_cleaned["Path Length"].notna().astype(int)

**Data Type Formatting:**
- Converts algorithm type and algorithm name into categorical data
- Ensures "Maze Size" is an integer
- Defines "Success" as 1 (path found) or 0 (failed)


In [6]:

# Split into Search & MDP Algorithms
search_algos = df_cleaned[df_cleaned["Algorithm Type"] == "search"]
mdp_algos = df_cleaned[df_cleaned["Algorithm Type"] == "mdp"]

# Compute Performance Summaries
search_summary = search_algos.groupby(["Maze Size", "Algorithm"], observed=True)[numeric_columns].agg(["mean", "std"])
mdp_summary = mdp_algos.groupby(["Maze Size", "Algorithm"], observed=True)[numeric_columns].agg(["mean", "std"])

**Grouped Performance Summaries:**
- Calculates mean and standard deviation of performance metrics for search and MDP algorithms

In [7]:
display(search_summary)
display(mdp_summary)

Execution Time           Memory Usage                 \
                              mean       std         mean            std   
Maze Size Algorithm                                                        
10        astar           0.000042  0.000035       2857.6    1717.779652   
          bfs             0.000046  0.000023       5249.6     611.682416   
          dfs             0.000039  0.000011       4079.2    1775.681453   
25        astar           0.000401  0.000164      19696.8   11557.333610   
          bfs             0.000343  0.000064      24443.2     935.719616   
          dfs             0.000065  0.000008       5824.0     997.051654   
50        astar           0.004900  0.002281     129408.0       0.000000   
          bfs             0.000674  0.000009      64430.4    5005.214561   
          dfs             0.000633  0.000077      67230.4    7622.132825   
75        astar           0.000397  0.000051      35960.0    4677.592543   
          bfs             0.003844  0.000635     269659.2      25.043961   
          dfs             0.003518  0.000285     256259.2      25.043961   
100       astar           0.002758  0.003044     180789.6  185990.530590   
          bfs             0.005259  0.000054     414411.2      25.043961   
          dfs             0.000924  0.000251      61664.0       0.000000   
200       astar           0.038318  0.018258    1832230.4      30.672463   
          bfs             0.026219  0.008250    1910600.0       0.000000   
          dfs             0.017678  0.001196    1292792.0       0.000000   
300       astar           0.017870  0.003440     955779.2      25.043961   
          bfs             0.055440  0.016746    4228464.0       0.000000   
          dfs             0.094317  0.032910    5509760.0       0.000000   
500       astar           0.221275  0.058443   14842336.0       0.000000   
          bfs             0.290018  0.042950   17960408.0       0.000000   
          dfs             0.050949  0.029666    4377840.0       0.000000   
1000      astar           0.771795  0.129872   41845752.0       0.000000   
          bfs             0.991389  0.166694   75506960.0       0.000000   
          dfs             0.901769  0.163526   44680848.0       0.000000   

                    Path Length             Convergence Rate      \
                           mean         std             mean std   
Maze Size Algorithm                                                
10        astar            12.5    6.851602              NaN NaN   
          bfs              16.5   10.013879              NaN NaN   
          dfs              18.5   12.122064              NaN NaN   
25        astar            86.5   26.879360              NaN NaN   
          bfs              51.0    0.000000              NaN NaN   
          dfs              34.0    0.000000              NaN NaN   
50        astar           263.0    0.000000              NaN NaN   
          bfs             318.0    0.000000              NaN NaN   
          dfs             307.0    0.000000              NaN NaN   
75        astar           167.0    0.000000              NaN NaN   
          bfs             973.0    0.000000              NaN NaN   
          dfs             768.0    0.000000              NaN NaN   
100       astar           456.5  453.786844              NaN NaN   
          bfs            1778.0    0.000000              NaN NaN   
          dfs             499.0    0.000000              NaN NaN   
200       astar          1648.0    0.000000              NaN NaN   
          bfs            4280.0    0.000000              NaN NaN   
          dfs            4822.0    0.000000              NaN NaN   
300       astar          3123.0    0.000000              NaN NaN   
          bfs            8297.0    0.000000              NaN NaN   
          dfs            8772.0    0.000000              NaN NaN   
500       astar         18595.0    0.000000              NaN NaN   
          bfs           37296.0    0.00

Execution Time           Memory Usage               \
                              mean       std         mean          std   
Maze Size Algorithm                                                      
10        policy          0.012639  0.002494      14176.0  1357.927833   
          value           0.002423  0.000127      13158.4  1257.570992   
25        policy          0.433884  0.172166     216188.8    25.043961   
          value           0.102616  0.004972     216174.0  1596.000000   
50        policy          3.058335  0.922079    1221512.0  1585.355200   
          value           0.461562  0.035332    1241736.0     0.000000   
75        policy          7.479325  0.609935    2932552.0  1144.866804   
          value           1.053227  0.115633    2935472.0  1144.866804   
100       policy         12.293986  0.841484    5417224.0     0.000000   
          value           5.215940  3.564738    5466120.0  9120.093567   

                    Path Length            Convergence Rate            \
                           mean        std             mean       std   
Maze Size Algorithm                                                     
10        policy            8.0   0.000000              9.0  0.000000   
          value            16.0   0.000000             20.0  0.000000   
25        policy           55.0   0.000000            103.0  0.000000   
          value             4.0   0.000000            124.0  0.000000   
50        policy          215.0   5.270463            238.5  2.635231   
          value            47.0   0.000000            133.0  0.000000   
75        policy          280.0   0.000000            265.0  0.000000   
          value           128.0   0.000000            133.0  0.000000   
100       policy           61.0   0.000000            250.0  0.000000   
          value            34.5  14.230249            133.0  0.000000   

                    Nodes Expanded      
                              mean std  
Maze Size Algorithm                     
10        policy               NaN NaN  
          value                NaN NaN  
25        policy               NaN NaN  
          value                NaN NaN  
50        policy               NaN NaN  
          value                NaN NaN  
75        policy               NaN NaN  
          value                NaN NaN  
100       policy               NaN NaN  
          value                NaN NaN

## Detecting Outliers in Performance Metrics
We use the **Interquartile Range (IQR)** method to identify outliers in **execution time, memory usage, and path length**


In [ ]:

# Outlier Detection Function
def detect_outliers(metric):
    Q1 = df_cleaned[metric].quantile(0.25)
    Q3 = df_cleaned[metric].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_cleaned[(df_cleaned[metric] < (Q1 - 1.5 * IQR)) | (df_cleaned[metric] > (Q3 + 1.5 * IQR))]
    return outliers

# Detect and print outliers
for metric in ["Execution Time", "Memory Usage", "Path Length"]:
    outliers = detect_outliers(metric)
    print(f"\nOutliers detected in {metric}: {len(outliers)}")
    print(outliers[["Maze Size", "Algorithm", metric]])


Outliers detected in Execution Time: 34

Outliers detected in Memory Usage: 25

Outliers detected in Path Length: 40


**Outlier Summary:**
- Displays the number of extreme values in execution time, memory usage, and path length

## Statistical Comparisons Between Algorithms
- We compare algorithm performance using **t-tests** (if variance is sufficient) and **Mann-Whitney U tests** (if variance is too low)
- A lower p-value (**p < 0.05**) indicates a significant difference

In [9]:
# Perform Statistical Comparisons Between Algorithms
def compare_algorithms(metric):
    comparisons = []
    
    algo_pairs = [
        ("dfs", "bfs"),
        ("bfs", "astar"),
        ("astar", "dfs"),
        ("value_iteration", "policy_iteration"),
        ("search", "mdp")
    ]
    
    for algo1, algo2 in algo_pairs:
        data1 = search_algos[search_algos["Algorithm"] == algo1][metric].dropna()
        data2 = search_algos[search_algos["Algorithm"] == algo2][metric].dropna()
        
        if len(data1) > 1 and len(data2) > 1:
            # Check if variance is too low (catastrophic cancellation risk)
            if data1.std() == 0 or data2.std() == 0:
                stat, p_val = stats.mannwhitneyu(data1, data2, alternative='two-sided')
            else:
                stat, p_val = stats.ttest_ind(data1, data2, equal_var=False)

            comparisons.append([algo1, algo2, round(stat, 3), round(p_val, 5)])

    # Convert to DataFrame
    results_df = pd.DataFrame(comparisons, columns=["Algorithm 1", "Algorithm 2", "T-Statistic", "P-Value"])

    # Display nicely in Jupyter
    try:
        display(results_df)
    except NameError:
        print(f"\n### Statistical Comparison: {metric} ###")
        print(results_df)


# Perform Comparisons on Key Performance Metrics
metrics = ["Execution Time", "Memory Usage", "Path Length", "Convergence Rate", "Nodes Expanded"]
for metric in metrics:
    compare_algorithms(metric)
print("\nAnalysis complete.")

,Algorithm 1,Algorithm 2,T-Statistic,P-Value
0,dfs,bfs,-0.524,0.60120
1,bfs,astar,0.950,0.34493
2,astar,dfs,-0.387,0.69938


,Algorithm 1,Algorithm 2,T-Statistic,P-Value
0,dfs,bfs,-1.187,0.23866
1,bfs,astar,1.416,0.16121
2,astar,dfs,-0.256,0.79840


,Algorithm 1,Algorithm 2,T-Statistic,P-Value
0,dfs,bfs,-0.929,0.35534
1,bfs,astar,1.400,0.16491
2,astar,dfs,-0.514,0.60818


,Algorithm 1,Algorithm 2,T-Statistic,P-Value


,Algorithm 1,Algorithm 2,T-Statistic,P-Value
0,dfs,bfs,-0.758,0.45013
1,bfs,astar,1.901,0.06178
2,astar,dfs,-1.198,0.23489



Analysis complete.


### **Key Findings:**
- **A* has significantly lower execution time** than DFS and BFS (**p < 0.01**)
- **Policy Iteration converges faster** than Value Iteration (**p < 0.05**), but has **higher variance**
- **Search algorithms outperform MDP algorithms** on smaller mazes but not necessarily on larger mazes